In [35]:
%reload_ext autoreload
%autoreload 2

from mlp import MlpRun

In [36]:
LBP_PATH = "./data/lbp"
CLASS_NAMES = ["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]

In [37]:
# Best Params: {'batch_size': 32, 'lr': 0.001, 'module__dropout_rate': 0.2, 'module__hidden_sizes': [256, 128]}
# Best Params: {'batch_size': 128, 'lr': 0.001, 'module__dropout_rate': 0.2, 'module__hidden_sizes': [256, 128]}
# Best Params: {'batch_size': 32, 'lr': 0.001, 'module__dropout_rate': 0.2, 'module__hidden_sizes': [128]}
# Best Params: {'batch_size': 32, 'lr': 0.001, 'module__dropout_rate': 0.3, 'module__hidden_sizes': [256, 128, 64]}

batch_size = 128
learning_rate = 0.01
dropout_rate = 0.2
hidden_sizes = [256, 128]
epochs = 100
output_size = 4  # Four classes

In [38]:
run = MlpRun(LBP_PATH, CLASS_NAMES, batch_size, hidden_sizes, output_size, dropout_rate, learning_rate)

Features dtype: torch.float32
Features shape: torch.Size([21165, 256])
Labels dtype: torch.int64
Labels shape: torch.Size([21165])


In [39]:
run.train(epochs)

[0 Training loss: 522.9739470481873
[0 Val loss: 1.004434928075591
[1 Training loss: 463.5345099568367
[1 Val loss: 0.7535589461895957
[2 Training loss: 437.9395218491554
[2 Val loss: 0.7235929788048587
[3 Training loss: 425.87103885412216
[3 Val loss: 0.755731626677869
[4 Training loss: 414.66462248563766
[4 Val loss: 0.7493079576029706
[5 Training loss: 408.82182213664055
[5 Val loss: 0.6854945705897773
[6 Training loss: 401.97105208039284
[6 Val loss: 0.7040906214891974
[7 Training loss: 400.9363763332367
[7 Val loss: 0.7353770047871034
[8 Training loss: 394.253152012825
[8 Val loss: 0.9478326084008858
[9 Training loss: 392.95456725358963
[9 Val loss: 0.6904833814991054
[10 Training loss: 388.6796897351742
[10 Val loss: 0.7870190997621906
[11 Training loss: 386.1788448393345
[11 Val loss: 0.7098412171228609
[12 Training loss: 385.2091920077801
[12 Val loss: 0.773212894130109
[13 Training loss: 383.70613411068916
[13 Val loss: 0.6834936426646674
[14 Training loss: 377.9678103327751
[

accuracy,▃▆▇▆▆▆▇▆▇▇▇▂▆▇▁▆▇▇▆▇▄▇██▇▇▇▇▅▆██▅▇██▇███
precision,▃▆▇▆▆▆▇▆▇▇▇▂▆▇▁▆▇▇▆▇▄▇██▇▇▇▇▅▆██▅▇██▇███
recall,▃▆▇▆▆▆▇▆▇▇▇▂▆▇▁▆▇▇▆▇▄▇██▇▇▇▇▅▆██▅▇██▇███
val_loss,▆▃▂▃▃▃▂▂▂▂▂█▂▂▇▃▂▃▄▂▄▂▁▁▂▂▂▁▃▃▁▁▃▁▁▁▁▁▁▁
accuracy,0.74905
precision,0.74905
recall,0.74905
val_loss,0.6116


In [40]:
run.test()